In [ ]:
%matplotlib inline
import numpy as np
import sys
from pathlib import Path

module_path = Path('../..')
if module_path not in sys.path:
    sys.path.append(str(module_path.resolve()))
    
from itertools import product
import mf2
import multiLevelCoSurrogates as mlcs

np.random.seed(20160501)  # Setting seed for reproducibility
OD = mf2.forrester

np.set_printoptions(linewidth=200, edgeitems=10, precision=4, suppress=True)
plot_dir = Path('../../plots/')
data_dir = Path('../../files/')

from function_defs import *

# Testing models on 2D version of 1D function

In [ ]:
# %%writefile -a function_defs.py

# defining some point styles
red_dot = {'marker': '.', 'color': 'red'}
blue_circle = {'marker': 'o', 'facecolors': 'none', 'color': 'blue'}

In [ ]:
# %%writefile -a function_defs.py

@mf2.row_vectorize
def td_inv_high(xx):
    x1, x2 = xx
    return -(OD.high(x1) + OD.high(x2))

@mf2.row_vectorize
def td_inv_low(xx):
    x1, x2 = xx
    return -(OD.low(x1) + OD.low(x2))


TD_inv = mf2.MultiFidelityFunction(
    name="Two dimensional inverted forrester",
    u_bound=np.array(np.repeat(OD.u_bound, 2)), l_bound=np.array(np.repeat(OD.l_bound, 2)),
    functions=[td_inv_high, td_inv_low],
    fidelity_names=['high', 'low'],
)

In [ ]:
u_bound, l_bound = TD_inv.u_bound, TD_inv.l_bound
steps = [.025, .025]
surf_high = mlcs.createsurface(TD_inv.high, u_bound=u_bound, l_bound=l_bound, step=steps)
surf_low = mlcs.createsurface(TD_inv.low, u_bound=u_bound, l_bound=l_bound, step=steps)

In [ ]:
mlcs.plotsurfaces([surf_high, surf_low], titles=['High fidelity (2d)', 'Low fidelity (2d)'], save_as=f'{plot_dir}2d_high_low_true_mesh.pdf')

In [ ]:
mlcs.plotsurfaces([surf_high, surf_low], as_3d=False, titles=['High fidelity (2d)', 'Low fidelity (2d)'], save_as=f'{plot_dir}2d_high_low_true.pdf')

## Basic modelling

In [ ]:
n_low = 16
n_high = 6

In [ ]:
# %%writefile -a function_defs.py

def create_models_and_compare(func, low, high, save_as=None):
    archive = mlcs.CandidateArchive(ndim=2, fidelities=['high', 'low', 'high-low'])
    archive.addcandidates(low, func.low(low), fidelity='low')
    archive.addcandidates(high, func.high(high), fidelity='high')

    mfbo = mlcs.MultiFidelityBO(func, archive, schema=[1,1])

    surf_high_model = mlcs.createsurface(mfbo.models['high'].predict, u_bound=u_bound, l_bound=l_bound, step=steps)
    surf_low_model = mlcs.createsurface(mfbo.models['low'].predict, u_bound=u_bound, l_bound=l_bound, step=steps)

    points_high = [mlcs.ScatterPoints(*archive.getcandidates(fidelity='high'), red_dot)]
    points_low = [mlcs.ScatterPoints(*archive.getcandidates(fidelity='low'), blue_circle)]

    points = [
        points_high, points_low,
        points_high, points_low,
    ]

    mlcs.plotsurfaces([surf_high, surf_low, surf_high_model, surf_low_model], shape=(2,2), 
                      titles=['high', 'low', 'high (hierarchical model)', 'low (model)'], all_points=points,
                      save_as=save_as)

As a first attempt, let's simply use all 2d combinations of the coordinates originally used for the example figure

In [ ]:
low_x = np.linspace(0,1,6).reshape((-1,1))
high_x = low_x[[2,3]].reshape((-1,1))

high_xy = np.array(list(product(high_x.flatten(), repeat=2)))
low_xy =  np.array(list(product(low_x.flatten(), repeat=2)))

create_models_and_compare(TD_inv, low_xy, high_xy, save_as=f'{plot_dir}2d_grid_trained_models.pdf')

### With a random sample

In [ ]:
np.random.seed(20160501)

low_xy = low_random_sample(ndim=2, nlow=n_low)
high_xy = low_xy[np.random.choice(n_low, n_high, replace=False)]

create_models_and_compare(TD_inv, low_xy, high_xy, save_as=f'{plot_dir}2d_random_trained_models.pdf')

### With an LHS

In [ ]:
np.random.seed(20160501)

low_xy = low_lhs_sample(ndim=2, nlow=n_low)
high_xy = low_xy[np.random.choice(n_low, n_high, replace=False)]

create_models_and_compare(TD_inv, low_xy, high_xy, save_as=f'{plot_dir}2d_LHS_trained_models.pdf')